[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sunfish256/fitbit-api-python/blob/main/app/get_heart_rate.ipynb)

In [ ]:
!pip install fitbit

In [ ]:
'''Module providing the function to read and write json files.'''
import json
import datetime
import fitbit
import pandas as pd

In [ ]:
# Settings
TOKEN_FILE_PATH = './fb_conf.json'
DATE = 'today'  # Your region's date, not GMT
FREQ = '1sec'

In [ ]:
def hr2csv(token_file_path, date, freq):
    '''
    Returns Fitbit heart rate data for a given date in data frames.
    In addition, outputs the data frame to Fitbit_HR_yyyy-MM-dd.csv.

    Args:
        token_file_path (str): file path
        date (str): yyyy-MM-dd or today
        freq (str): 1sec | 1min | 5min | 15min

    Returns:
        DataFrame: index='timestamp', columns='heartrate'
    '''
    if date == 'today':
        date = str(datetime.date.today())
    # Read credentials
    with open(token_file_path, 'r', encoding='utf-8') as f:
        conf = json.load(f)

    client_id = conf['client_id']
    client_secret = conf['client_secret']
    access_token = conf['access_token']
    refresh_token = conf['refresh_token']
    # Initialize client
    client = fitbit.Fitbit(
        client_id=client_id,
        client_secret=client_secret,
        access_token=access_token,
        refresh_token=refresh_token,
    )
    # Get heart rate data
    act_data = client.intraday_time_series(
        resource='activities/heart',
        base_date=date,
        detail_level=freq
    )
    heartrate = act_data['activities-heart-intraday']['dataset']
    heartrate_df = pd.DataFrame.from_dict(heartrate)
    heartrate_df.insert(0, 'timestamp', pd.to_datetime(date + ' ' + heartrate_df['time']))
    heartrate_df.drop('time', axis=1, inplace=True)
    heartrate_df.rename(columns={'value': 'heartrate'}, inplace=True)
    # Export to csv
    heartrate_df.to_csv(f'Fitbit_HR_{DATE}.csv', index=False, encoding='utf-8')
    return heartrate_df


if __name__ == '__main__':
    res = hr2csv(TOKEN_FILE_PATH, DATE, FREQ)
    print(res)